In [30]:
import requests
import json
import pandas as pd

'''
Missing in AB:
owner
roomnumber
workshift
'''
INPUT_TABLES = ['actionbuilder_orgtx']
INPUT_SCHEMA = 'action_builder_api'

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class ActionBuilderAPIClient:
    def __init__(self):
        # API #
        self.API_KEY = 'qKAMp-xAALpfgzxuIe7RFA'
        self.CUSTOM_AB_ENDPOINT = 'aft'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'OSDI-API-Token':self.API_KEY}
        self.POST_HEADERS = {'OSDI-API-Token':self.API_KEY, 'Content-Type': 'application/json'}
        self.API_URL = f'https://{self.CUSTOM_AB_ENDPOINT}.actionbuilder.org/api/rest/v1/'

        # INTERNAL #
        self.campaign_list = {}
        
    def loader(self):
        camps = self.getResource('campaigns')
        for camp in camps['_embedded']['action_builder:campaigns']:
            
            self.campaign_list.update({camp['name'].lower():camp['identifiers'][0]})
        
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getResource(self, resource):
        return self.getJSON(resource)
        
client = ActionBuilderAPIClient()
client.loader()

def upload_individuals(payload, campaign_name, campaign_id):
    print(f"Running Upload for Campaign {campaign_name} (ID: {campaign_id})")
    print(f"Found {len(payload)} updates...")
    print("-------------------------------")
    for input_payload in payload:  
        out = {
            "person": {
                "action_builder:entity_type": "Person",
                "given_name": input_payload['firstname'],
                "family_name": input_payload['lastname'],
                "additional_name":input_payload['middlename'],
                "action_builder:nickname":input_payload['preferredname'],
                "postal_addresses":[
                    {
                        "address_lines":[input_payload['address']],
                        "locality":input_payload['city'],
                        "region":input_payload['state'],
                        "postal_code":input_payload['zip']
                    }
                ],
                "email_addresses": [

                ],
                "phone_numbers": [

                ],  

                "identifiers": ["custom_id:"+input_payload['individualguid']]
              },
            "add_tags":[
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Chapter",
                    "action_builder:name":input_payload['chaptername']
                },
                {
                    "action_builder:section":"7 - Union Role",
                    "action_builder:field":"Officer (Make note of the local role name)",
                    "action_builder:name":input_payload['officerrolename']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Employer",
                    "action_builder:name":input_payload['employername']
                },
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Is Working?",
                    "action_builder:name":input_payload['isworking']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Authorizer",
                    "action_builder:name":input_payload['authorizer']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Charter Holder",
                    "action_builder:name":input_payload['charterholder']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Management Firm",
                    "action_builder:name":input_payload['managementfirm']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Department",
                    "action_builder:name":input_payload['department']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Building",
                    "action_builder:name":input_payload['building']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Campus",
                    "action_builder:name":input_payload['campus']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"College",
                    "action_builder:name":input_payload['college']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Program",
                    "action_builder:name":input_payload['program']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Work Area",
                    "action_builder:name":input_payload['workarea']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Work Region",
                    "action_builder:name":input_payload['workregion']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Work Unit",
                    "action_builder:name":input_payload['workunit']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Worksite",
                    "action_builder:name":input_payload['worksite']
                },
                {
                    "action_builder:section":"5 - Job Info",
                    "action_builder:field":"Floor",
                    "action_builder:name":input_payload['floor']
                },
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Unit",
                    "action_builder:name":input_payload['unit']
                },
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Local Dues Categories",
                    "action_builder:name":input_payload['localduescategoryname']
                },
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Local",
                    "action_builder:name":input_payload['affiliatenumber']
                },
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Local Dues Payment Method",
                    "action_builder:name":input_payload['paymentmethodname']
                },
                {
                    "action_builder:section":"6 - Union Relationship Details",
                    "action_builder:field":"Union Relationship (DO NOT EDIT)",
                    "action_builder:name":input_payload['unionrelationship']
                }
            ]
        }
        ljc_out = {
            "action_builder:section":"5 - Job Info",
            "action_builder:field":"Local Job Class",
            "action_builder:name":input_payload['localjobclassname']
        }
        if input_payload['jobdescription']:
            ljc_out.update({"action_builder:note_response":input_payload['jobdescription']})
        out['add_tags'].append(ljc_out)

        unirole_out = {
            "action_builder:section":"7 - Union Role",
            "action_builder:field":"Committee Type",
            "action_builder:name":input_payload['committeetype']
        }
        if input_payload['committeemembertype']:
            unirole_out.update({"action_builder:note_response":input_payload['committeemembertype']})
        out['add_tags'].append(unirole_out)


        if str(input_payload['addressstatus']) == "4":
            out['person']['postal_addresses'][0].update({"status:":"verified"})


        
        print(f"-Uploading individual with guid: {input_payload['individualguid']} and name {input_payload['firstname']} {input_payload['lastname']}")
        ab_ref = client.getResource(f"campaigns/{campaign_id}/people?filter=identifier eq 'custom_id:{input_payload['individualguid']}'")

        phone_list = []
        email_list = []
        
        if input_payload['mobilephone']:
            input_payload['mobilephone'] = "1" + input_payload['mobilephone']
        if input_payload['homephone']:
            input_payload['homephone'] = "1" + input_payload['homephone']
                
        if ab_ref['total_pages'] != 0:
            print("Found individual, updating existing person record...")
            current_ab_ref = ab_ref['_embedded']['osdi:people'][0]
            current_ab_ref_id = current_ab_ref['identifiers'][0].replace('action_builder:','')

            if 'phone_numbers' in current_ab_ref:
                ab_ref_phones = current_ab_ref['phone_numbers']
                for nums in ab_ref_phones:
                    phone_list.append(nums['number'])

            if 'email_addresses' in current_ab_ref:
                ab_ref_emails = current_ab_ref['email_addresses']
                for emails in ab_ref_emails:
                    email_list.append(emails['address'])

            if input_payload['unionrelationship']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Union Relationship (DO NOT EDIT)'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Union Relationship tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))

            if input_payload['agency']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Agency'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Agency tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))

            if input_payload['authorizer']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Authorizer'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Authorizer tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                        
            if input_payload['building']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Building'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Building tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['campus']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Campus'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Campus tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                        
            if input_payload['charterholder']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Charter Holder'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Charter Holder tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['college']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'College'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing College tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['department']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Department'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Department tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['employername']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Employer'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Employer tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['floor']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Floor'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Floor tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['localjobclassname']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Local Job Class'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing LocalJobClass tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['program']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Program'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Program tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['workarea']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Work Area'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing WorkArea tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['workregion']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Work Region'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing WorkRegion tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['workunit']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Work Unit'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing WorkUnit tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['worksite']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Worksite'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Worksite tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                        
            if input_payload['chaptername']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Chapter'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Chapter tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
            
            if input_payload['managementfirm']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Management Firm'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Management Firm tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
            
            if input_payload['isworking']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Is Working?'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing isWorking tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['affiliatenumber']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Local'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Local tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                        
            if input_payload['localduescategoryname']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Local Dues Categories'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing LocalDuesCategories tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
                    
            if input_payload['unit']:
                tag_ref = client.getResource(f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings?filter=action_builder:field eq 'Unit'")
                if tag_ref['total_pages'] != 0:
                    print("--Found existing Unit tag, deleting for overwrite...")
                    r = requests.delete(url = client.API_URL + f"campaigns/{campaign_id}/people/{current_ab_ref_id}/taggings/{tag_ref['_embedded']['osdi:taggings'][0]['identifiers'][0].replace('action_builder:','')}", headers = client.GET_HEADERS)
                    if r.status_code != 200 and r.status_code != 201:
                        print(json.dumps(r.json(), indent=4))
        else:
            print("No matching person record found, creating new record...")
            
        if input_payload['mobilephone']:
            if not input_payload['mobilephone'] in phone_list:
                if str(input_payload['mobilephonestatus']) == '4':
                    out['person']['phone_numbers'].append({"number": input_payload['mobilephone'], "number_type": "Cell", "action_builder:confirmed":"verified"})
                else:
                    out['person']['phone_numbers'].append({"number": input_payload['mobilephone'], "number_type": "Cell"})
            
        if input_payload['homephone']:
            if not input_payload['homephone'] in phone_list:
                if str(input_payload['homephonestatus']) == '4':
                    out['person']['phone_numbers'].append({"number": input_payload['homephone'], "number_type": "Home", "action_builder:confirmed":"verified"})
                else:
                    out['person']['phone_numbers'].append({"number": input_payload['homephone'], "number_type": "Home"})

        if input_payload['workemail']:
            if not input_payload['workemail'].lower() in email_list:
                if str(input_payload['workemailstatus']) == '4':
                    out['person']['email_addresses'].append({"address": input_payload['workemail'], "address_type": "Work", "action_builder:confirmed":"verified"})
                else:
                    out['person']['email_addresses'].append({"address": input_payload['workemail'], "address_type": "Work"})
            
        if input_payload['homeemail']:
            if not input_payload['homeemail'].lower() in email_list:
                if str(input_payload['homeemailstatus']) == '4':
                    out['person']['email_addresses'].append({"address": input_payload['homeemail'], "address_type": "Home", "action_builder:confirmed":"verified"})
                else:
                    out['person']['email_addresses'].append({"address": input_payload['homeemail'], "address_type": "Home"})

        r = requests.post(url = client.API_URL + f'campaigns/{campaign_id}/people', headers = client.POST_HEADERS, data = json.dumps(out))
        jprint(out)
        print(r)
        if r.status_code != 200 and r.status_code != 201:
            print(json.dumps(r.json(), indent=4))
        else:
            print("-Person upload complete.")
            print("---------------------------")  





test_data = [{
    "individualguid":"99999-9943-40D9-9999-999999888",
    "affiliatenumber":"Orgtx",
    "lastname":"Tester",
    "firstname":"API",
    "middlename":"Mid",
    "preferredname":"TST",
    "chaptername":"TFT Training Local Structured Chapter",
    "officerrolename":"",
    "localjobclassname":"(Training PSRP) Office Specialist",
    "employername":"Training PSRP Employer",
    "authorizer":"Unknown",
    "charterholder":"Unknown",
    "managementfirm":"Unknown",
    "committeemembertype":"",
    "committeetype":"",
    "jobdescription":"Admin Assisstant",
    "isworking":"Yes",
    "agency":"Unknown",
    "department":"Unknown",
    "building":"Unknown",
    "campus":"Unknown",
    "college":"Unknown",
    "program":"Unknown",
    "workarea":"Unknown",
    "workregion":"Unknown",
    "workunit":"Unknown",
    "worksite":"Unknown",
    "floor":"Unknown",
    "unit":"ORGTX PSRP Test Unit",
    "localduescategoryname":"Half Dues",
    "paymentmethodname":"Unknown",
    "unionrelationship":"Member",
    
    "address":"12 Tsrt Rd",
    "city":"Fairfax",
    "individualaddresstypename":"",
    "state":"VA",
    "zip":"22013",
    "addressstatus":"4",
    
    "workemail":"Trainingemail19878@email.com",
    "homeemail":"",
    "workemailstatus":"4",
    "homeemailstatus":"4",
    "mobilephonestatus":"4",
    "homephonestatus":"4",
    "mobilephone":"",
    "homephone":"3334447870"

}]

upload_individuals(test_data,"OrgTX - Training","179ddc9a-288d-4426-9849-b4975e98118d")

# r = requests.get(url = client.API_URL + f'campaigns/179ddc9a-288d-4426-9849-b4975e98118d/people/16336de2-df1b-4dad-8d27-516a0f1b56ea', headers = client.GET_HEADERS)
# print(json.dumps(r.json(), indent=4))

Running Upload for Campaign OrgTX - Training (ID: 179ddc9a-288d-4426-9849-b4975e98118d)
Found 1 updates...
-------------------------------
-Uploading individual with guid: 99999-9943-40D9-9999-999999888 and name API Tester
No matching person record found, creating new record...
{
    "person": {
        "action_builder:entity_type": "Person",
        "given_name": "API",
        "family_name": "Tester",
        "additional_name": "Mid",
        "action_builder:nickname": "TST",
        "postal_addresses": [
            {
                "address_lines": [
                    "12 Tsrt Rd"
                ],
                "locality": "Fairfax",
                "region": "VA",
                "postal_code": "22013",
                "status:": "verified"
            }
        ],
        "email_addresses": [
            {
                "address": "Trainingemail19878@email.com",
                "address_type": "Work",
                "action_builder:confirmed": "verified"
            }
 